- **데이터 불러오기 기능** : data_load()
- **파라미터와 편향 생성 기능** : initialization_parameter()
- **데이터 뒤섞기 기능** : data_shuffle()
- **학습 데이터 분리 기능** : train_data_split()
- **테스트 데이터 분리 기능** : test_data_split()
- **미니배치를 고려한 학습 데이터 기반의 신경망 연산 및 이진 판단 예측 기능** : model_train()
- **손실(교차 엔트로피)값 연산 기능** : cross_entropy_loss()
- **정확도 연산 기능** : calculate_accuracy()

In [ ]:
import numpy as np
import pandas as pd
import csv
import matplotlib.pyplot as plt

In [ ]:
def data_load(path):
  """
    데이터 불러오는 함수

        Args:
            path (str) : colab 업로드 된 파일 경로
            
        Retruns:
            data (DataFrame) : binary_dataset.csv
  """
  data = pd.read_csv(path)
  return data

In [ ]:
def initialization_parameter(inputs):
  """
    가중치 및 편향 생성 함수

        Args:
            inputs (int) : X_train의 열 갯수 -> 독립변수 x1~x8 8개
            
        Retruns:
            w1 (array) : numpy를 통해 생성된 [0,1]의 범위의 난수를 갖는 8x1 배열
            b1 (array) : 0으로 채워진 1차원 배열
  """
  np.random.seed(40)

  w1 = np.random.randn(inputs,1)
  b1 = np.zeros(1,)

  return w1, b1

In [ ]:
def data_shuffle(data):
  """
    DataFrame 행을 무작위로 섞는 함수

        Args:
            data (DataFrame) : 기존 불러왔던 binary_dataset.csv
            
        Retruns:
            shuffled_data (DataFrame) : binary_dataset.csv의 행의 순서를 무작위로 섞은 후의 데이터 프레임
  """
  shuffled_data = data.sample(frac=1).reset_index(drop=True)

  return shuffled_data

In [ ]:
def train_data_split(data, size=0.8):
  """
    학습 데이터 분리 함수

        Args:
            data (DataFrame) : data_shuffle에서 반환 된 데이터 프레임
            size (int) : 학습 데이터 분리 비중 설정

        Retruns:
            X_train (DataFrame) : 독립 변수 y를 제외한 index 0~15에 해당하는 16x8 데이터 프레임
            y_train (Series) : index 0~15에 해당하는 종속 변수 y 값 16개
            
        Note:
            data.shape[0] : 기존 데이터 프레임의 행의 갯수 20개
            data_train : data[:16] 기존 데이터 index 0~15에 해당하는 값 슬라이싱
  """
  num = int(data.shape[0]*size)
  data_train = data[:num]
  X_train = data_train.drop(['y'], axis=1)
  y_train = data_train['y']

  return X_train, y_train

In [ ]:
def test_data_split(data, size = 0.2):
  """
    테스트 데이터 분리 함수

        Args:
            data (DataFrame) : data_shuffle에서 반환 된 데이터 프레임
            size (int) : 테스트 데이터 분리 비중 설정

        Retruns:
            X_test (DataFrame) : 독립 변수 y를 제외한 index 0~3에 해당하는 4x8 데이터 프레임
            y_test (Series) : index 0~3에 해당하는 종속 변수 y 값 4개
            
        Note:
            data.shape[0] : 기존 데이터 프레임의 행의 갯수 20개
            data_train : data[-4:] 기존 데이터 index 16~19에 해당하는 값 슬라이싱
  """
  num = int(data.shape[0]*size)
  data_test = data[-num:]
  X_test = data_test.drop(['y'], axis=1).reset_index(drop=True)
  y_test = data_test['y'].reset_index(drop=True)

  return X_test, y_test

In [ ]:
def model_train(X, y, w1, b1, batch_size):
  """
    미니배치를 고려한 학습 데이터 기반의 신경망 연산 및 이진 판단 예측 기능 함수

        Args:
            X (DataFrame) : 순전파 진행 시 사용 될 독립 변수 데이터 프레임
            y (int) : 순전파 진행 시 사용 될 종속 변수 데이터 프레임
            w1 (array) : initialization_parameter 함수를 통해 생성된 가중치
            b1 (array) : initialization_parameter 함수를 통해 생성된 편향
            batch_size (int) : 사용자 지정 batch_size

        Retruns:
            np.mean(accuracy) (float) : calculate_accuracy 함수를 통해 각 minibatch 마다 계산된 accuracy를 평균 낸 값
            np.mean(cross_entropy) (float) : cross_entropy_loss 함수를 통해 각 minibatch 마다 계산된 loss를 평균 낸 값
            
        Note:
            a1 : batch_size에 따른 학습 데이터 분리 후 가중치와 내적, 편향 추가
            output : sigmoid 활성화 함수 적용 후 2차원 배열 -> 1차원 배열로 변환 (확률값)
            sigmoid_output : output 중 0.5 이상 값은 1로 이하는 0으로 변환 한 배열
  """
  accuracy = []
  cross_entropy = []

  for i in range(0, len(X), batch_size):
    minibatch = X[i:i+batch_size]  
    a1 = np.dot(minibatch, w1) + b1
    output = np.concatenate(sigmoid(a1).tolist())
    sigmoid_output = list(map(lambda x : 1 if x>=0.5 else 0, output))
  
    accuracy.append(calculate_accuracy(sigmoid_output, y[i:i+batch_size]))
    cross_entropy.append(cross_entropy_loss(output, y[i:i+batch_size]))

  return np.mean(accuracy), np.mean(cross_entropy)

In [ ]:
def sigmoid(s):
  """
    sigmoid 활성화 함수를 구현한 함수

        Args:
            s (array) : 가중치-편향 연산된 가중합 a1
            
        Retruns:
            1 / (1+np.exp(-s)) (array)
  """
  return 1 / (1+np.exp(-s))

In [ ]:
def calculate_accuracy(y_pred, y_true):
  """
    정확도 연산 기능 구현 함수

        Args:
            y_pred (list) : batch_size 별 순전파를 진행한 모델의 예측 값(1 또는 0)을 담은 sigmoid_output 배열
            y_true (Series) : batch_size 별 실제 타겟 값

        Retruns:
            minibatch_accuracy (float) : batch_size 별 정확도 평균 값
            
        Note:
            check_match : y_pred와 y_true의 일치 여부 확인 후 일치하는 수 만큼 cnt 증가
  """
  cnt = 0
  minibatch_accuracy = 0
  check_match = np.equal(y_pred,y_true)

  for correct in check_match:
    if correct:
      cnt+=1

  minibatch_accuracy = cnt/len(y_pred)

  return minibatch_accuracy

In [ ]:
def cross_entropy_loss(y_pred, y_true):
  """
    손실(교차 엔트로피)값 연산 기능 구현 함수
    
        Args:
            y_pred (array) : batch_size 별 순전파를 진행한 모델의 확률 값을 담은 output 배열
            y_true (Series) : batch_size 별 실제 타겟 값

        Retruns:
            second (float) : batch_size 별 loss 평균 값
  """
  first = y_true * np.log(y_pred + 1e-7)
  second = -np.sum(first)

  return second

In [ ]:
def main():
  data = data_load('/content/binary_dataset.csv')
  data = data_shuffle(data)
  X_train, y_train = train_data_split(data)
  X_test, y_test = test_data_split(data)
  w1, b1 = initialization_parameter(X_train.shape[1])
  accuracy, loss = model_train(X_train, y_train, w1, b1, 4)
  print(f"[Epoch 1] TrainData - Loss = {loss}, Accuracy = {accuracy}")

In [ ]:
main()